# Ввод и вывод. Работа с файлами

## Потоки ввода и вывода

В языках высокого уровня практически все операции ввода-вывода абстрагируются до отправки или получения данных из *потоков ввода и вывода* (I/O streams).

Основные операции с потоками:
* запись байта в поток вывода
* чтение байта из потока ввода
* просмотр следующего байта без удаления его из потока
* проверка, что поток не находится в специальном состоянии "конец файла" (EOF)

В минимальном случае поток поддерживает только последовательные операции. Для отдельных случаев возможны операции по произвольному адресу ("установка курсора" -- `seek`). В первую очередь, это возможно для потоков, связанных с файлами на дисках.

При чтении символа он в общем случае удаляется из потока. Это позволяет работать с потоками, не связанными с файлами на диске, -- поток ввода с клавиатуры, поток ввода из сети, с датчика и т.п.

**EOF**
"Конец файла" -- специальное состояние, означающее, что из него ничего нельзя больше прочитать. *В отличие от конца строки, конец файла -- это не символ*.

В Unix-системах есть специальный файл `/dev/null`, который всегда находится в состоянии EOF.

При вводе с клавиатуры ситуация конца файла эмулируется нажатием Ctrl+D (полезно, если работаем с функцией, которая читает ввод до конца файла, но хотим передавать в неё данные с клавиатуры).

## Файловые потоки

Чаще всего потоки связываются с файлами (в Unix-системах вообще практически всё является "файлом" -- устройства, таблицы процессов и т.д.)

Общая схема работы с файлом:

1. Открытие файла создаёт новый поток ввода-вывода, связываем его с идентификатором
```python
stream = open(filename)
```

2. Работа "с файлом" (реально с потоком)
```python
work_with(stream)
```

3. Закрытие потока
```python
close(stream)
```

Если поток не закрыть:
* в Windows -- связанный с ним файл может оказаться недоступен для чтения или редактирования
* в Unix -- операции записи могут не завершиться (фактическая запись будет только в буфер ОС, а до диска не дойдёт)

### Работа с файлами с учётом возможности ошибок

Одна из характерных причин, почему файл не закрылся -- ошибка во время обработки. Рассмотрим функцию, которая читает числа из одного файла по два за раз и пишет в другой числа и их частное

```python
def print_ratios(in_file, out_file):
    in_stream = open(in_file)
    out_stream = open(out_file)
    while True:
        x_str, y_str = in_stream.readline(), in_stream.readline()
        x_str and y_str or break # остановиться если дошли до конца файла
        x, y = float(x_str), float(y_str)
        print(x, y, x / y, file=out_stream)
        
    close(in_stream)
    close(out_stream)
    return
```

Здесь всё, казалось бы, нормально -- файлы открываются в начале, закрываются в конце. Но до конца функция может не дойти, если в ходе обработки будет ошибка -- во входном файле будут записаны строки, не приводящиеся к числам, или знаменатель в одной из пар окажется нулём.

В таких случаях используется идиома `try-finally`:
```python
stream = open(filename)
try:
    work_with(stream)
finally:
    close(stream)
```

Помещение работы с потоком в блок `try` означает, что мы допускаем возможность ошибки. Помещение закрытия потока в блок `finally` означает, что операция должна выполниться в конце, даже если в блоке `try` произошла ошибка.

### Менеджер контекста

Поскольку такая идиома часто встречается, в Python встроены специальные менеджеры контекста. Для файлов это будет

```python
with open(filename) as file_stream:
    work_with(file_stream)
```

Такой синтаксис автоматически оборачивает работу с файлом в указанный выше блок.

### Реализация менеджера контекста

Конструкция `with expr as alias` является синтаксическим сахаром для примерно такого кода:

```python
alias = expr
alias.__enter__()
try:
    work_with(alias)
finally:
    alias.__exit__()
```

Конструкция `with-as`, таким образом, будет работать с любыми объектами, для которых должным образом определены методы `__enter__` и `__exit__`. Подробное описание см. в [документации Python](https://docs.python.org/3/library/stdtypes.html#typecontextmanager) и [PEP 343](https://www.python.org/dev/peps/pep-0343/).

## Потоки ввода-вывода, не связанные с файлами

Поскольку поток ввода-вывода должен лишь реализовывать некоторый интерфейс, он не обязан быть связан с файлом на диске.

В качестве примера рассмотрим `StringIO`.

In [1]:
from io import StringIO

In [16]:
string_stream = StringIO(
"""abc
def
ijk""")

In [17]:
for ln in string_stream:
    print(ln)

abc

def

ijk


`StringIO` ведёт себя как обычный поток ввода-вывода, но находится полностью в памяти.

Его удобно применять для тестирования:
* вместо чтения данных из файла можно задать тестовые данные строкой и обернуть её в `StringIO`
* вместо вывода в файл или на экран можно выводить в `StringIO` и потом просмотреть строку

**Пример** (из Advent of Code 2017 https://adventofcode.com/2017/day/2)

На вход подана таблица чисел, записанная в файле. Нужно найти её "контрольную сумму" - сумму разностей между максимальным и минимальным значением в каждой строке.

Например, для таблицы
```
5 1 9 5
7 5 3
2 4 6 8
```
контрольная сумма равна (9 - 1) + (7 - 3) + (8 - 2) = 18.

**Решение**. Начнём с обработки строки

In [4]:
def row_checksum(str):
    nums = [int(a) for a in str.split()]
    return max(nums) - min(nums)

Далее найдём сумму, предполагая чтение из потока:

In [5]:
def checksum(iostream):
    s = 0
    for ln in iostream:
        s += row_checksum(ln)
    return s

Протестируем на предложенном примере:

In [18]:
def test_checksum():
    test_io = StringIO(
'''5 1 9 5
7 5 3
2 4 6 8''')
    
    if checksum(test_io) == 18:
        print("Test passed")
    else:
        print("Test failed")

In [19]:
test_checksum()

Test passed


Для чтения из файла можно использовать
```python
def checksum_file(path)
    with open(path) as input:
        return checksum(input)
```

## Буферизация ввода-вывода

Поскольку доступ к дисковым или сетевым устройствам медленный по сравнению с доступом к памяти, операции обычно *буферизуются* -- чтение и запись идут большими кусками в буфер в памяти, откуда по необходимости выдаются программе или сбрасываются на диск по заполнению буфера записи.

Буферизация может происходить на нескольких уровнях -- на уровне библиотеки времени исполнения, операционной системы, дисковых и сетевых устройств.

При работе с дисковыми устройствами с большой задержкой (напр., сетевые диски) может быть выгоднее читать файлы целиком в память и затем работать с данными в памяти, чем читать с сетевого диска построчно. То же самое относится и к записи.

### Сброс потока (flush)

Операции записи типа `print` по умолчанию буферизуются, т.е. записываемые данные временно сохраняются в памяти. При неожиданном завершении программы (в C или C++ такое может произойти из-за ошибки работы с памятью) буфер "умирает" вместе с программой, и данные не доходят до устройства ввода-вывода. Если мы хотим какую-то информацию информацию гарантированно скинуть на устройство, можно для команды вывода указать необходимость операции flush -- сброса внутреннего буфера в поток.

В Python для функции `print` можно указать необязательный аргумент `flush=True`. В C++ перенос строки через `endl` сбрасывает буфер, тогда как простой вывод `\n` не сбрасывает.

С точки зрения производительности, буфер лучше сбрасывать как можно реже, т.к. сброс -- это выполнение как минимум системного вызова и ожидание, пока все буферизованные данные передадутся на устройство вывода. Однако сброс буфера лучше делать всегда при печати отладочных сообщений, иначе при падении программы они могут не дойти до вывода.